<p> Neste Notebook eu calculo a similaridade entre os nomes dos times ingleses presentes na Base FIFA e os times ingleses que disputam a Premier League presentes na Base BetExplorer. Algumas correções são feitas na base proveniente do BetExplorer para corrigir abreviações e nicknames, estas alterações foram feitas baseadas nos dados presentes em <a href="https://en.wikipedia.org/wiki/List_of_football_club_nicknames_in_the_United_Kingdom">List of football club nicknames in the United Kingdom</a>.</p>

<p> Após a correção é feito o cálculo de similaridade de Jaccard e uma nova validação manual foi realizada para casos onde a similaridade calculada foi menor que 50%.</p>

<p> As bases são então atualizadas com o ID de cada time.</p>

In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
from nltk import word_tokenize as wt
import matplotlib.pyplot as plt

<p>Após uma breve análise dos dados é possível perceber um padrão presente nos nomes dos times ingleses que é a presença frequente das palavras <b>United, City </b> e <b>Town</b>, o que pode gerar um grande impacto no cálculo de Jaccard visto que os nomes dos times possuem poucos palavras, variando entre 1 e 3 palavras. Visto isso foi realizada uma alteração no algoritmo onde casos onde a única similaridade encontrada era entre uma destas três palavras, a similaridade era então modificada para zero. A alteração foi feita para evitar missmatchs.</p>

In [23]:
def jaccard_similarity(label1, label2):
    label1 = wt(label1)
    label2 = wt(label2)
    if len(set(label1).intersection(set(label2)) - set(set(label1).intersection(set(label2))).intersection(set(['United','City','Town']))) == 0:
        return 0
    return len(set(label1).intersection(set(label2)))/len(set(label1).union(set(label2)))

def removerAcentosECaracteresEspeciais(colunas):
    retorno = []
    for palavra in colunas:
        # Unicode normalize transforma um caracter em seu equivalente em latin.
        nfkd = unicodedata.normalize('NFKD', palavra)
        palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])

        # Usa expressão regular para retornar a palavra apenas com números, letras e espaço
        retorno.append(re.sub('[^a-zA-Z0-9\\\]', ' ', palavraSemAcento))
    return retorno

def tratamentoNome(column):
    column = column.str.replace("FC","").str.replace("F C","")
    column = column.str.replace("Utd","United")
    return column

In [56]:
df_fifa = pd.read_csv('data/FIFA_CONSOLIDADO4.csv')
df_games = pd.read_csv('data/Games_Results.csv')
df_nicknames = pd.read_csv('data/England_NickNames.csv')

In [57]:
df_teams_id = df_fifa[df_fifa.Team_Country == 'England'][['Teams', 'Team_Country']].drop_duplicates().sort_values(by='Teams').reset_index(drop=True)
df_teams_id = df_teams_id.rename_axis('TeamID').reset_index()
df_fifa_england = df_fifa[df_fifa.Team_Country == 'England']
del df_fifa

<p>Através de validação manual com os dados presentes neste <a href="https://en.wikipedia.org/wiki/List_of_football_club_nicknames_in_the_United_Kingdom">link</a> é possível fazer a substituição abaixo.</p>

In [58]:
#Apenas para a Premier League
df_games = df_games[(df_games.Tournament == 'Premier League') & (df_games.Country == 'England')]
df_games.drop('Fase', axis=1,inplace=True)
df_games.loc[df_games.Home == 'QPR', 'Home'] = 'Queens Park Rangers'
df_games.loc[df_games.Away == 'QPR', 'Away'] = 'Queens Park Rangers'
df_games.loc[df_games.Home == 'Wolves', 'Home'] = 'Wolverhampton Wanderers'
df_games.loc[df_games.Away == 'Wolves', 'Away'] = 'Wolverhampton Wanderers'
df_games['Home_Team_ID'] = 0.0
df_games['Away_Team_ID'] = 0.0

In [59]:
home = df_games.Home.drop_duplicates()
away = df_games.Away.drop_duplicates()

In [60]:
df_fifa_teams = df_teams_id
df_fifa_teams['Perfect_Match'] = 0
df_integracao = pd.DataFrame({'Team':pd.concat([home,away]).drop_duplicates().reset_index(drop = True)})
df_integracao['Similaridade'] = 0.0
df_integracao['Match_Team'] = ''
df_integracao['TeamID'] = np.NaN
df_fifa_teams['Team_Sim'] = removerAcentosECaracteresEspeciais(df_fifa_teams.Teams)
df_integracao['Team_Sim'] = removerAcentosECaracteresEspeciais(df_integracao.Team)

In [61]:
%%time
for id1, team1 in df_integracao.iterrows():
    for id2, team2 in df_fifa_teams.iterrows():
        if team2[-2] == 0:
            sim = jaccard_similarity(team1[-1],team2[-1])
            if sim > df_integracao.loc[id1,'Similaridade'].item() or (sim == df_integracao.loc[id1,'Similaridade'].item() and team2.str.contains(team1[-1])[-1]):
                df_integracao.loc[id1,'Similaridade'] = sim
                df_integracao.loc[id1,'Match_Team'] = df_fifa_teams.loc[id2,'Teams']
                df_integracao.loc[id1,'TeamID'] = df_fifa_teams.loc[id2,'TeamID']
                if sim == 1:
                    df_fifa_teams.loc[id2, 'Perfect_Match'] = 1

Wall time: 9.09 s


In [62]:
df_integracao.sort_values(by='Similaridade', ascending=False, inplace=True)
df_integracao

,Team,Similaridade,Match_Team,TeamID,Team_Sim
0,Everton,1.000000,Everton,39.0,Everton
13,Liverpool,1.000000,Liverpool,54.0,Liverpool
30,Portsmouth,1.000000,Portsmouth,75.0,Portsmouth
25,Fulham,1.000000,Fulham,42.0,Fulham
24,Manchester City,1.000000,Manchester City,57.0,Manchester City
1,Manchester United,1.000000,Manchester United,58.0,Manchester United
22,Sunderland,1.000000,Sunderland,90.0,Sunderland
20,Watford,1.000000,Watford,97.0,Watford
32,Crystal Palace,1.000000,Crystal Palace,33.0,Crystal Palace
16,Chelsea,1.000000,Chelsea,25.0,Chelsea


In [63]:
df_games.columns

Index(['GameID', 'Country', 'Tournament', 'Season', 'Round', 'Home', 'Away',
       'Score', 'Bet_Home', 'Bet_Drawn', 'Bet_Away', 'Game_Date',
       'Game_Details', 'Home_Team_ID', 'Away_Team_ID'],
      dtype='object')

In [64]:
df_games = pd.merge(left=df_games, right=df_integracao[['Team','Match_Team','TeamID']], left_on=['Home'], right_on=['Team'], how='left')
df_games[['Home', 'Home_Team_ID']] = df_games.loc[df_games.Match_Team.notnull(), ['Match_Team','TeamID']]
#np.where(df_games.Match_Team.notnull(), df_games[['Match_Team','TeamID']], df_games[['Home','Home_Team_ID']])
df_games.drop(['Match_Team','TeamID','Team'], axis = 1, inplace=True)

df_games = pd.merge(left=df_games, right=df_integracao[['Team','Match_Team','TeamID']], left_on=['Away'], right_on=['Team'], how='left')
df_games[['Away', 'Away_Team_ID']] = df_games.loc[df_games.Match_Team.notnull(), ['Match_Team','TeamID']]
#= np.where(df_games.Match_Team.notnull(), df_games[['Match_Team','TeamID']], df_games[['Home','Home_Team_ID']])
df_games.drop(['Match_Team','TeamID','Team'], axis = 1, inplace=True)

df_games

,GameID,Country,Tournament,Season,Round,Home,Away,Score,Bet_Home,Bet_Drawn,Bet_Away,Game_Date,Game_Details,Home_Team_ID,Away_Team_ID
0,377825,England,Premier League,1998/1999,1. Round,Everton,Aston Villa,0:0,3.10,2.95,2.29,15.08.1998,/soccer/england/premier-league-1998-1999/evert...,39.0,4.0
1,377826,England,Premier League,1998/1999,1. Round,Manchester United,Leicester City,2:2,1.60,2.90,6.00,15.08.1998,/soccer/england/premier-league-1998-1999/manch...,58.0,51.0
2,377827,England,Premier League,1998/1999,1. Round,Newcastle United,Charlton Athletic,0:0,1.60,2.90,6.00,15.08.1998,/soccer/england/premier-league-1998-1999/newca...,64.0,24.0
3,377828,England,Premier League,1998/1999,1. Round,Middlesbrough,Leeds United,0:0,2.70,2.70,2.70,15.08.1998,/soccer/england/premier-league-1998-1999/middl...,60.0,50.0
4,377829,England,Premier League,1998/1999,1. Round,Sheffield Wednesday,West Ham United,0:1,2.10,2.85,3.50,15.08.1998,/soccer/england/premier-league-1998-1999/sheff...,83.0,99.0
5,377830,England,Premier League,1998/1999,1. Round,AFC Wimbledon,Tottenham Hotspur,3:1,2.50,2.70,2.90,15.08.1998,/soccer/england/premier-league-1998-1999/wimbl...,0.0,94.0
6,377831,England,Premier League,1998/1999,1. Round,Blackburn Rovers,Derby County,0:0,1.90,3.00,4.09,15.08.1998,/soccer/england/premier-league-1998-1999/black...,8.0,36.0
7,377832,England,Premier League,1998/1999,1. Round,Coventry City,Chelsea,2:1,2.90,2.80,2.50,15.08.1998,/soccer/england/premier-league-1998-1999/coven...,30.0,25.0
8,377833,England,Premier League,1998/1999,1. Round,Southampton,Liverpool,1:2,3.30,2.90,2.29,16.08.1998,/soccer/england/premier-league-1998-1999/south...,85.0,54.0
9,377834,England,Premier League,1998/1999,1. Round,Arsenal,Nottingham Forest,2:1,1.35,4.09,9.00,17.08.1998,/soccer/england/premier-league-1998-1999/arsen...,3.0,68.0


In [65]:
df_games[['Home_Goals','Away_Goals']] = pd.DataFrame(df_games.Score.str.split(':',1).tolist(), columns = ['Home_Goals','Away_Goals'])
df_games.drop('Score', axis=1, inplace=True)

In [66]:
df_integracao

,Team,Similaridade,Match_Team,TeamID,Team_Sim
0,Everton,1.000000,Everton,39.0,Everton
13,Liverpool,1.000000,Liverpool,54.0,Liverpool
30,Portsmouth,1.000000,Portsmouth,75.0,Portsmouth
25,Fulham,1.000000,Fulham,42.0,Fulham
24,Manchester City,1.000000,Manchester City,57.0,Manchester City
1,Manchester United,1.000000,Manchester United,58.0,Manchester United
22,Sunderland,1.000000,Sunderland,90.0,Sunderland
20,Watford,1.000000,Watford,97.0,Watford
32,Crystal Palace,1.000000,Crystal Palace,33.0,Crystal Palace
16,Chelsea,1.000000,Chelsea,25.0,Chelsea


In [71]:
df_fifa_premier = pd.merge(left=df_fifa_england, right=df_integracao[['Match_Team','TeamID']], left_on= ['Teams'], right_on=['Match_Team'])
df_fifa_premier.drop(['Match_Team'], axis=1, inplace=True)

In [73]:
df_fifa_premier.to_csv('data/FIFA_Premier_League_Teams.csv', index=False)

In [75]:
df_games.to_csv('data/BET_Premier_League_Games.csv', index=False)